In [1]:
import tensorflow as tf
import pandas as pd
import requests
import json
import base64

In [ ]:
df = pd.read_csv("data/spam.csv")
test_text = df["v2"][0]
test_text

In [3]:
def prepare_json(text):
    feature_spec = {
        "text": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, "utf-8")]))
    }

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]

    return json.dumps({
        "signature_name": "serving_default",
        "instances": result
    })

In [4]:
def make_predictions(text):
    json_data = prepare_json(text)

    endpoint = "http://localhost:8080/v1/models/spam-detection-model:predict"
    response = requests.post(endpoint, data=json_data)

    prediction = response.json()["predictions"][0][0]
    return "Not Spam" if prediction > 0.6 else "Spam"

In [ ]:
make_predictions(test_text)